In [1]:
%matplotlib inline

In [2]:
import numpy as np
import math
from numpy import expand_dims
from scipy.integrate import odeint,quad,dblquad,simps,quad_vec,nquad

In [3]:
import tensorflow as tf

import keras
from keras import backend as K
from keras import datasets, layers, models
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import optimizers
from keras.callbacks import ModelCheckpoint

from matplotlib import pyplot
import pandas as pd
from glob import glob 
import csv

from sklearn.model_selection import train_test_split
from skimage.io import imread

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from colorspacious import cspace_converter
from collections import OrderedDict

import pylab as py
import threading
import scipy

from PIL import Image

from scipy import constants
from scipy import interpolate
from scipy import signal
from scipy import ndimage
from scipy.linalg import block_diag
import sympy

from matplotlib.image import imread
import skimage
from skimage.transform import rotate
from skimage.io import imshow, show, imsave
from skimage import img_as_ubyte


Using TensorFlow backend.


In [4]:
# concatenate the CSV Files of the 
train_labels = sorted(glob('/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/labels/label*.csv'))
N_label = len(train_labels)
# write header of csv file
header = ['id','appliance']
with open('data_cnn/header.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)

# create dataset
fout = open('data_cnn/labels.csv','w')

for line in open('data_cnn/header.csv'):
    fout.write(line)

for i in range(N_label):
    file = open(train_labels[i])
#     file.__next__()
    for line in file:
        fout.write(line)
    file.close()
fout.close()
# ######### weg got our dataset train_labels ##########

In [6]:
# Read the Dataset we created without data augmentation ####
train_df_aug = pd.read_csv('data_cnn/labels.csv', index_col=0)
train_df_aug['id'] = train_df_aug.index.map(lambda id: id)
train_df_aug['amplitude'] = train_df_aug.index.map(lambda id: f'data_cnn/target/{id}_ampl.png')
train_df_aug['phase'] = train_df_aug.index.map(lambda id: f'data_cnn/target/{id}_phase.png')
# print(len(train_df))
# ftraindf = open('data_cnn/train_df.csv','a')
# ftraindf.write(train_df)
len(train_df_aug.amplitude.values)

8

In [7]:
for i in range(len(train_df_aug)):
    phase_aug = imread(train_df_aug.phase.values[i]) ## read the train images (ampl,phase)
    ampl_aug = imread(train_df_aug.amplitude.values[i])
    id_aug = train_df_aug.id[i] 
    appl_aug = train_df_aug.appliance[i] ## read id and appliance to correlate
    
    ### rotate the images (ampl,phase) and save them
    for k in range(3):
        sess = tf.InteractiveSession()
        phase_rot1 = tf.contrib.image.rotate(phase_aug, 90*(k+1)*np.pi/180, interpolation='bilinear')
        phase_rot1 = phase_rot1.eval() ## rotate the image
        ampl_rot1 = tf.contrib.image.rotate(ampl_aug, 90*(k+1)*np.pi/180, interpolation='bilinear')
        ampl_rot1 = ampl_rot1.eval()

        imsave('data_cnn/target_aug/'+id_aug+'_rot'+str(k)+'_ampl.png', img_as_ubyte(ampl_rot1)) ##save the images in the train file
        imsave('data_cnn/target_aug/'+id_aug+'_rot'+str(k)+'_phase.png', img_as_ubyte(phase_rot1))
    
    
        label_aug = [id_aug+'_rot'+str(k),  appl_aug]
        with open(('/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label_aug/label_'+id_aug+'_rot'+str(k)+'.csv'), 'w') as fid:
            writer = csv.writer(fid)
            writer.writerow(label_aug) ## write the correlated labels for the augmented data images
    
    
    sess = tf.InteractiveSession()
    phase_flip_1 = tf.image.flip_left_right(phase_aug)
    phase_flip_1 = phase_flip_1.eval()
    ampl_flip_1 = tf.image.flip_left_right(ampl_aug)
    ampl_flip_1 = ampl_flip_1.eval()
    phase_flip_2 = tf.image.flip_up_down(phase_aug)
    phase_flip_2 = phase_flip_2.eval()
    ampl_flip_2 = tf.image.flip_up_down(ampl_aug)
    ampl_flip_2 = ampl_flip_2.eval()
    
    
    imsave('data_cnn/target_aug/'+id_aug+'_flip_lr_ampl.png', img_as_ubyte(ampl_flip_1))
    imsave('data_cnn/target_aug/'+id_aug+'_flip_ud_ampl.png', img_as_ubyte(ampl_flip_2))
    
    imsave('data_cnn/target_aug/'+id_aug+'_flip_lr_phase.png', img_as_ubyte(phase_flip_1))
    imsave('data_cnn/target_aug/'+id_aug+'_flip_ud_phase.png', img_as_ubyte(phase_flip_2))
    
    
       
    label_flip_lr = [id_aug+'_flip_lr',  appl_aug]
    with open(('/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label_aug/label_'+id_aug+'_flip_lr.csv'), 'w') as fid:
        writer = csv.writer(fid)
        writer.writerow(label_flip_lr) ## write the correlated labels for the augmented data images
                   
    label_flip_ud = [id_aug+'_flip_ud',  appl_aug]
    with open(('/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label_aug/label_'+id_aug+'_flip_ud.csv'), 'w') as fid2:
        writer = csv.writer(fid2)
        writer.writerow(label_flip_ud) ## write the correlated labels for the augmented data images
         

In [8]:
# concatenate the CSV Files of the 
train_labels = sorted(glob('/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label_aug/label*.csv'))
N_label = len(train_labels)
# write header of csv file
header = ['id','appliance']
with open('data_cnn/header.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)

# create dataset
fout = open('data_cnn/labels.csv','w')

for line in open('data_cnn/header.csv'):
    fout.write(line)

for i in range(N_label):
    file = open(train_labels[i])
#     file.__next__()
    for line in file:
        fout.write(line)
    file.close()
fout.close()
# ######### weg got our dataset train_labels ##########